Site para entender a chamada de informações pela API de Dados Abertos da Câmara: https://www.gustavofurtado.com/DadosAbertosBrasil/camara.html

### Importações


In [1]:
!pip install -q -U google-generativeai

In [2]:
!pip install -q -U DadosAbertosBrasil

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 479.7 kB/s eta 0:00:00


In [3]:
import google.generativeai as genai
import requests
import pandas as pd
from DadosAbertosBrasil import camara

### Inicializações

In [16]:
# Substitua pelas suas credenciais
GOOGLE_API_KEY="SUA_API_KEY"
genai.configure(api_key=GOOGLE_API_KEY)

# Parâmetros gerais
generation_config = {
    "candidate_count": 1, # Candidatos são as possibilidades de respostas
    "temperature": 0.2, # Quanto mais próximo de 1, mais criativas serão as respostas (mais variação)
    #"max_output_tokens": 150 # Manterá as respostas do chat mais concisas
}

# Parâmetros de segurança
safety_settings = {
    "HARASSMENT": "BLOCK_LOW_AND_ABOVE",
    "HATE": "BLOCK_LOW_AND_ABOVE",
    "SEXUAL": "BLOCK_LOW_AND_ABOVE",
    "DANGEROUS": "BLOCK_LOW_AND_ABOVE"
}

# Modelo
model = genai.GenerativeModel(model_name = "gemini-1.0-pro",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

# Inicializando o histórico do chat
chat = model.start_chat(history=[])

# Última mensagem padrão do Bot
last_bot_message = None

# Variáveis Gerais
duvidas = False

### Funções Mensagens Padrão

In [17]:
# Funções

# Estrutura Modelo de Resposta Padrão
def response():
  pass # colocar aqui dps

# Mensagem Padrão Menu
def opcoes_menu():
  print("""
  Digite um dos números das opções abaixo para saber mais:\n
  1. Blocos Partidários e Partidos\n
  2. Deputados\n
  3. Comissões e Órgãos\n
  4. Propostas Parlamentares\n
  5. Dúvidas Gerais\n
  """)

  last_bot_message = "opcoes_menu" # Informa última mensagem padrão enviada pelo Bot

# Mensagem Padrão Blocos Partidários
def opcoes_blocos():
  print("""
  Digite um dos números das opções abaixo para saber mais:\n
  1. Lista de Blocos Partidários\n
  2. Lista Partidos
  3. Dúvidas Gerais\n\n

  Para retornar às opções principais digite "menu"
  """)

  last_bot_message = "opcoes_blocos" # Informa última mensagem padrão enviada pelo Bot

# Mensagem Padrão Deputados
def opcoes_deputados():
  print("""
  Digite um dos números das opções abaixo para saber mais:\n
  1. Lista de Deputados por Estado\n
  2. Informações sobre Deputado Específico\n
  3. Despesas de Deputados por Estado\n
  4. Despesas de Deputado Específico
  5. Dúvidas Gerais\n\n

  Para retornar às opções principais digite "menu"
  """)

  last_bot_message = "opcoes_deputados" # Informa última mensagem padrão enviada pelo Bot

# Mensagem Padrão Comissões e Órgãos
def opcoes_orgaos():
  print("""
  Digite um dos números das opções abaixo para saber mais:\n
  1. Lista de Comissões e Órgãos\n
  2. Votações em Comissão ou Órgão Específico\n
  3. Eventos previstos em Comissão ou Órgão Específico\n
  4. Dúvidas Gerais\n\n

  Para retornar às opções principais digite "menu"
  """)

  last_bot_message = "opcoes_orgaos" # Informa última mensagem padrão enviada pelo Bot

# Mensagem Padrão Propostas Parlamentares
def opcoes_propostas():
  print("""
  Digite um dos números das opções abaixo para saber mais:\n
  1. Lista de Propostas Ativas\n
  2. Informações sobre Proposta Específica\n
  3. Histórico de Tramitação de Proposta Específica\n
  4. Dúvidas Gerais\n\n

  Para retornar às opções principais digite "menu"
  """)

  last_bot_message = "opcoes_propostas" # Informa última mensagem padrão enviada pelo Bot

### Funções Organizar Tabelas

In [49]:
# Organiza tabela de partidos
def tabela_partidos():
  dataframe_partidos = camara.lista_partidos()
  partidos = dataframe_partidos[['sigla', 'nome']]

  lista_situacao_partidos = []
  lista_total_posse_partidos = []
  lista_total_membros_partidos = []
  lista_lideres_partidos = []

  for index, row in dataframe_partidos.iterrows():
    codigo = row["codigo"]
    partido = camara.Partido(codigo)

    situacao_partido = partido.situacao
    lista_situacao_partidos.append(situacao_partido)

    total_posse_partido = partido.total_posse
    lista_total_posse_partidos.append(total_posse_partido)

    total_membros_partido = partido.total_membros
    lista_total_membros_partidos.append(total_membros_partido)

    lider_partido = partido.lider['nome']
    lista_lideres_partidos.append(lider_partido)

  df_info_partidos = pd.DataFrame({"status": lista_situacao_partidos, "lider": lista_lideres_partidos, "total posse": lista_total_posse_partidos, "total membros": lista_total_membros_partidos})
  partidos = pd.concat([partidos, df_info_partidos], axis=1)
  display(partidos)

In [51]:
# Organiza tabela de proposições (ADICIONAR AUTORES)
def tabela_proposicoes():
  proposicoes2024 = camara.lista_proposicoes(None, None, 2024)
  dataframe_proposicoes = proposicoes2024[['codigo', 'tipo', 'ano', 'ementa']]

  lista_temas = []

  for index, row in dataframe_proposicoes.iterrows():
    codigo = row["codigo"]
    proposicao = camara.Proposicao(codigo)

    lista_tema_proposicoes = proposicao.temas()

    temas_str = ""
    for index, row in lista_tema_proposicoes.iterrows():
      tema = row['tema']
      temas_str += tema + ", "

    temas_str = temas_str[:-2]
    lista_temas.append(temas_str)

  df_temas = pd.DataFrame({"temas": lista_temas})
  proposicoes = pd.concat([dataframe_proposicoes, df_temas], axis=1)

  display(proposicoes)

### Funções Opções

In [8]:
# Opção escolhida Menu
def num_menu(prompt):
  global duvidas

  if "1" in prompt: # Blocos Partidários e Partidos
    opcoes_blocos()
  elif "2" in prompt: # Deputados
    opcoes_deputados()
  elif "3" in prompt: # Comissões e Órgãos
    opcoes_orgaos()
  elif "4" in prompt: # Propostas Parlamentares
    opcoes_propostas()
  elif "5" in prompt: # Dúvidas Gerais
    print("""
    Digite a sua pergunta. Vai ser um prazer respondê-la!\n

    Para retornar às opções principais digite "menu"
    """)
    duvidas = True
  else:
    print("Digite o número desejado.")

# Opção escolhida Blocos Partidários
def num_blocos(prompt):
  global duvidas

  if "1" in prompt: # Lista de Blocos Partidários
    tabela_blocos = camara.lista_blocos()
    blocos = tabela_blocos[['nome', 'legislatura']]
    display(blocos)
  elif "2" in prompt: # Lista Partidos
    tabela_partidos()
  elif "3" in prompt:
    print("""
    Digite a sua pergunta. Vai ser um prazer respondê-la!\n

    Para retornar às opções principais digite "menu"
    """)
    duvidas = True
  else:
    print("Digite o número desejado.")

# Opção escolhida Deputados
def num_deputados(prompt):
  global duvidas

  if "1" in prompt: # Lista de Deputados por Estado
    estado = input('Digite a sigla do seu estado: ')

    try:
      lista_deputados = camara.lista_deputados(None, None, estado)
      deputados = lista_deputados[['nome', 'partido', 'uf', 'email']]
      display(deputados)
    except ValueError as e:
        print("Ocorreu um erro ao gerar a resposta. Tente novamente mais tarde.")

  elif "2" in prompt: # Informações sobre Deputado Específico
    pass
  elif "3" in prompt: # Despesas de Deputados por Estado
    pass
  elif "4" in prompt: # Despesas de Deputado Específico
    pass
  elif "5" in prompt: # Dúvidas Gerais
    print("""
    Digite a sua pergunta. Vai ser um prazer respondê-la!\n

    Para retornar às opções principais digite "menu"
    """)
    duvidas = True
  else:
    print("Digite o número desejado.")

# Opção escolhida Comissões e Órgãos
def num_orgaos(prompt):
  global duvidas

  if "1" in prompt:
    lista_orgaos = camara.lista_orgaos()
    orgaos = lista_orgaos[['nome', 'sigla', 'orgao_tipo']]
    display(orgaos)
  elif "2" in prompt:
    pass
  elif "3" in prompt:
    pass
  elif "4" in prompt:
    print("""
    Digite a sua pergunta. Vai ser um prazer respondê-la!\n

    Para retornar às opções principais digite "menu"
    """)
  else:
    print("Digite o número desejado.")

# Opção escolhida Propostas Parlamentares
def num_propostas(prompt):
  global duvidas

  if "1" in prompt: # Tabela de proposições
    tabela_proposicoes()
  elif "2" in prompt:
    pass
  elif "3" in prompt:
    pass
  elif "4" in prompt:
    print("""
    Digite a sua pergunta. Vai ser um prazer respondê-la!\n

    Para retornar às opções principais digite "menu"
    """)
    duvidas = True
  else:
    print("Digite o número desejado.")

### Main

**O que o usuário pode conferir:**
1. Lista de blocos partidários na atual legislação
2. Listagem e busca de deputados (pergunta se quer burcar especifico ou quer ver deputados do ESTADO)
- Se quiser especifico pede nome do deputado e fala se está em ação ou não
- Apresenta foto e informações gerais sobre o deputado
3. Despesas com o exercício parlamentar do deputado específico
4. Informações sobre comissões e órgãos da Câmara e seus Líderes
5. Informações detalhadas sobre votações em órgão específico da Câmara
6. Eventos para ocorrer em órgão específico da Câmara
7. Lista de proposições na Câmara e informações específicas de uma proposição
8. Histórico de passos da tramitação de uma proposta
9. Tipos de temas existentes e tipos de proposições existentes

In [ ]:
# Estabelecendo recebimento de prompt
opcoes_menu()
prompt = input("Faça sua pergunta: ")

# Enquanto o usuário não digitar "fim", sistema prossegue recebendo prompts
while prompt != "fim":

  if not duvidas:
    if "menu" in prompt:
      opcoes_menu()
    elif last_bot_message == "opcoes_menu":
      num_menu(prompt)
    elif last_bot_message == "opcoes_blocos":
      num_blocos(prompt)
    elif last_bot_message == "opcoes_deputados":
      num_deputados(prompt)
    elif last_bot_message == "opcoes_orgaos":
      num_orgaos(prompt)
    elif last_bot_message == "opcoes_propostas":
      num_propostas(prompt)

    if not duvidas:
      prompt = input("Faça sua pergunta: ")
  else:
    prompt = input("Faça sua pergunta: ")
    duvidas = False

    if "menu" not in prompt:
      try:
        response = chat.send_message(prompt)
        print("\nGuia Político: ", response.text, "\n")
      except ValueError as e:
        print("Ocorreu um erro ao gerar a resposta. Tente novamente mais tarde.")
        # Opcional: registrar o erro para análise posterior
        print(f"Erro: {e}")